<a href="https://colab.research.google.com/github/LuccasTraumer/ibjjf-data/blob/main/Worlds_IBJJF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import requests
import json
import re

# URL base dos JSONs
base_url = "https://raw.githubusercontent.com/LuccasTraumer/ibjjf-data/main/world-ibjjf/"
anos = list(range(1996, 2024))

# Mapeamento para normalizar faixas
mapa_faixas = {
    "white": "WHITE", "white belt": "WHITE",
    "blue": "BLUE", "blue belt": "BLUE",
    "purple": "PURPLE", "purple belt": "PURPLE",
    "brown": "BROWN", "brown belt": "BROWN",
    "black": "BLACK", "black belt": "BLACK",
    "coral": "CORAL", "coral belt": "CORAL",
    "red": "RED", "red belt": "RED"
}

# Mapeamento para normalizar sexo
mapa_sexo = {
    "male": "MALE", "m": "MALE", "masculino": "MALE",
    "female": "FEMALE", "f": "FEMALE", "feminino": "FEMALE"
}

dfs = []

for ano in anos:
    url = f"{base_url}{ano}.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)

        linhas = []
        for item in data:
            categoria_raw = item.get("categoria", "").strip()
            ranking = item.get("ranking", [])

            partes = [p.strip() for p in categoria_raw.split(" / ")]
            divisao = partes[0] if len(partes) > 0 else None
            faixa_raw = partes[1] if len(partes) > 1 else None
            sexo_raw = partes[2] if len(partes) > 2 else None
            categoria_peso = partes[3] if len(partes) > 3 else None

            # Normaliza faixa e sexo
            faixa = None
            sexo = None
            if faixa_raw:
                faixa_lower = faixa_raw.lower().replace("belt", "").strip()
                faixa = mapa_faixas.get(faixa_lower, faixa_raw.upper())
            if sexo_raw:
                sexo_lower = sexo_raw.lower().strip()
                sexo = mapa_sexo.get(sexo_lower, sexo_raw.upper())

            for posicao, atleta in enumerate(ranking, start=1):
                linhas.append({
                    "ano": ano,
                    "competicao": "World IBJJF",
                    "divisao": divisao,
                    "faixa": faixa,
                    "sexo": sexo,
                    "categoria_peso": categoria_peso,
                    "posicao": posicao,
                    "nome": atleta.get("nome"),
                    "academia": atleta.get("academia")
                })

        df = pd.DataFrame(linhas)
        dfs.append(df)
        print(f"✅ {ano} carregado — {len(df)} atletas")

    except Exception as e:
        print(f"⚠️ Erro ao carregar {ano}: {e}")

dados_bjj = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(dados_bjj)}")

# Preview dos dados
dados_bjj.head(10)


✅ 1996 carregado — 115 atletas
✅ 1997 carregado — 158 atletas
✅ 1998 carregado — 207 atletas
✅ 1999 carregado — 222 atletas
✅ 2000 carregado — 238 atletas
✅ 2001 carregado — 240 atletas
✅ 2002 carregado — 240 atletas
✅ 2003 carregado — 239 atletas
✅ 2004 carregado — 239 atletas
✅ 2005 carregado — 254 atletas
✅ 2006 carregado — 258 atletas
✅ 2007 carregado — 255 atletas
✅ 2008 carregado — 263 atletas
✅ 2009 carregado — 278 atletas
✅ 2010 carregado — 290 atletas
✅ 2011 carregado — 321 atletas
✅ 2012 carregado — 357 atletas
✅ 2013 carregado — 344 atletas
✅ 2014 carregado — 395 atletas
✅ 2015 carregado — 375 atletas
✅ 2016 carregado — 431 atletas
✅ 2017 carregado — 439 atletas
✅ 2018 carregado — 449 atletas
✅ 2019 carregado — 446 atletas
⚠️ Erro ao carregar 2020: 404 Client Error: Not Found for url: https://raw.githubusercontent.com/LuccasTraumer/ibjjf-data/main/world-ibjjf/2020.json
✅ 2021 carregado — 426 atletas
✅ 2022 carregado — 443 atletas
✅ 2023 carregado — 464 atletas

Total de regi

,ano,competicao,divisao,faixa,sexo,categoria_peso,posicao,nome,academia
0,1996,World IBJJF,ADULT,BLUE,MALE,ROOSTER,1,Willian Couto,
1,1996,World IBJJF,ADULT,BLUE,MALE,ROOSTER,1,Rodrigo Brandao,
2,1996,World IBJJF,ADULT,BLUE,MALE,ROOSTER,1,Alex Torres,
3,1996,World IBJJF,ADULT,BLUE,MALE,LIGHT FEATHER,1,Leonardo Santos,
4,1996,World IBJJF,ADULT,BLUE,MALE,LIGHT FEATHER,1,Carlos Maia,
5,1996,World IBJJF,ADULT,BLUE,MALE,LIGHT FEATHER,1,Adriano Lucio,
6,1996,World IBJJF,ADULT,BLUE,MALE,LIGHT FEATHER,1,Marcos Australiano,
7,1996,World IBJJF,ADULT,BLUE,MALE,FEATHER,1,Adriano Pereira,
8,1996,World IBJJF,ADULT,BLUE,MALE,FEATHER,1,Alexandre Simoes,
9,1996,World IBJJF,ADULT,BLUE,MALE,FEATHER,1,Aparecido Farias,


In [18]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(20)


,sexo,academia,titulos
1,BLACK,Alliance,39
11,BLACK,CheckMat,25
5,BLACK,Atos Jiu-Jitsu,24
19,BLACK,Gracie Barra,16
20,BLACK,Gracie Humaita,13
18,BLACK,GF Team,9
0,BLACK,AOJ,6
23,BLACK,Infight JJ,6
28,BLACK,PSLPB Cicero Costha,6
27,BLACK,Ns Brotherhood,5


In [7]:
# Atletas mais recorrentes
atletas = (
    dados_bjj.groupby("nome")
    .size()
    .sort_values(ascending=False)
)
atletas.head(10)


,0
nome,
Roger Gracie,17
Beatriz de Oliveira Mesquita,17
Ana Carolina Vieira Srour,16
Leandro Lo Pereira do Nascimento,15
Alexandre Ribeiro,15
Nicholas de Barcellos Meregali,14
Marcus Vinícius Oliveira de Almeida,14
Gabrieli Pessanha de Souza Marinho,14
Mayssa Caldas Pereira Bastos,13


In [8]:
# Evolução da quantidade de participantes por ano
import plotly.express as px

contagem_ano = (
    dados_bjj.groupby("ano")
    .size()
    .reset_index(name="quantidade")
)

px.bar(contagem_ano, x="ano", y="quantidade", title="Número de atletas registrados por ano")


In [13]:
# Total de participações por atleta (independente da colocação)
participacoes = (
    dados_bjj
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="participacoes")
    .sort_values(["sexo", "participacoes"], ascending=[True, False])
)

# Top 10 masculinos e femininos
top_masculino = participacoes[participacoes["sexo"] == "MALE"].head(10)
top_feminino  = participacoes[participacoes["sexo"] == "FEMALE"].head(10)

print("🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:")
display(top_masculino)

print("🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:")
display(top_feminino)


🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:


,sexo,nome,participacoes
4763,MALE,Roger Gracie,17
3079,MALE,Alexandre Ribeiro,15
3124,MALE,Andre Galvao,11
4748,MALE,Rodrigo Medeiros,11
4274,MALE,Marcelo Garcia,10
3261,MALE,Braulio Estima,9
4352,MALE,Mario Reis,9
4831,MALE,Saulo Ribeiro,9
4776,MALE,Romulo Barral,8
4819,MALE,Samuel Braga,8


🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:


,sexo,nome,participacoes
2705,FEMALE,Leticia Ribeiro,10
2693,FEMALE,Kyra Gracie,9
2797,FEMALE,Penny Thomas,8
2595,FEMALE,Fabiana Borges,6
2495,FEMALE,Beatriz Mesquita,5
2610,FEMALE,Gabrielle Garcia,5
2732,FEMALE,Mackenzie Dern,5
2762,FEMALE,Michelle Nicolini,5
2850,FEMALE,Silvana Abreu,5
2443,FEMALE,Alessandra Vieira,4


In [22]:
# Apenas os campeões (posição = 1)
campeoes = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)

# Top 10 de cada sexo
top_campeoes_masc = campeoes[campeoes["sexo"] == "MALE"].head(10)
top_campeoes_fem  = campeoes[campeoes["sexo"] == "FEMALE"].head(10)

print("🥇 Top 10 Campeões (Masculino):")
display(top_campeoes_masc)

print("🥇 Top 10 Campeãs (Feminino):")
display(top_campeoes_fem)


🥇 Top 10 Campeões (Masculino):


,sexo,nome,titulos
1732,MALE,Roger Gracie,12
962,MALE,Alexandre Ribeiro,9
983,MALE,Andre Galvao,7
1486,MALE,Marcelo Garcia,7
1227,MALE,Francisco Fernandes,6
993,MALE,Antonio Braga Neto,5
1048,MALE,Bruno Malfacine,5
1085,MALE,Celso Vinicius,5
1193,MALE,Fabio Gurgel,5
1497,MALE,Marcio Cruz,5


🥇 Top 10 Campeãs (Feminino):


,sexo,nome,titulos
825,FEMALE,Leticia Ribeiro,6
822,FEMALE,Kyra Gracie,4
831,FEMALE,Luzia Fernandes,4
833,FEMALE,Mackenzie Dern,4
841,FEMALE,Michelle Nicolini,4
735,FEMALE,Alessandra Vieira,3
738,FEMALE,Ana Carolina,3
742,FEMALE,Ana Laura Cordeiro,3
755,FEMALE,Bianca Andrade,3
793,FEMALE,Gabrielle Garcia,3


from matplotlib import pyplot as plt
top_campeoes_fem['titulos'].plot(kind='hist', bins=20, title='titulos')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
top_campeoes_fem['titulos'].plot(kind='line', figsize=(8, 4), title='titulos')
plt.gca().spines[['top', 'right']].set_visible(False)

In [15]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(10)


,sexo,academia,titulos
1,BLACK,Alliance,39
11,BLACK,CheckMat,25
5,BLACK,Atos Jiu-Jitsu,24
19,BLACK,Gracie Barra,16
20,BLACK,Gracie Humaita,13
18,BLACK,GF Team,9
0,BLACK,AOJ,6
23,BLACK,Infight JJ,6
28,BLACK,PSLPB Cicero Costha,6
27,BLACK,Ns Brotherhood,5


In [21]:
import plotly.express as px

px.bar(
    top_campeoes_masc,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeões (Masculino)",
    text="titulos"
)

px.bar(
    top_campeoes_fem,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeãs (Feminino)",
    text="titulos"
)


In [20]:
# Academias com mais campeões (1º lugar)
top_academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby("academia")
    .size()
    .sort_values(ascending=False)
    .head(10)
)
top_academias


,0
academia,
,702
Alliance,260
Gracie Barra,207
Atos Jiu-Jitsu,135
Gracie Humaita,109
CheckMat,86
GF Team,85
Team Lloyd Irvin,47
Brasa,33
